# Applied Data Science Capstone project

## Battle of Neighbourhood - week 1

## Exploring business opportunities in setting up Food joint by exploring venues of Banglore, India.

### 1. Introduction/ a description of the problem and a discussion of the background.

India being a food-loving country with each region having its own special cuisine, Indians have never been very big on eating out. But the scenario is changing now. The restaurant industry in India and in Banglore as well has been growing at a rapid pace over the past few years or so and the growth story is set to continue for the next foreseeable future. This project aims to explore venue related to food joints in various regions of Banglore city using Foursquare API. This project would help all those interested in setting up food joint such as Restaurent, Pizza Place, café etc. in various region pf Banglore city. This would help to analyse which type of food joint and its number are already there in a particular region so as to decide the type of food joint to set up in that particular region. 

### 2. A description of the Data and how it will be used to solve the problem.

#### The Data used and its source

I have scrapred the wikipedia page ('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore') and then used the FourSquare API to collect data about various regions and neighborhood locations of Banglore, India. Using the Foursquare API it was found that there are eight regions viz. Central, Eastern, North-Eastern, Northern, South-Eastern, Southern, Southern suburbs and Western region. The number of neighborhood were determined along its Latitude and Longitude coordinates to show various neighbourhoods in the map.

#### How the data is used to solve the problem?

I have determined latitude and longitude coordinate of each region and this was used to calculate the distance from the point to the food joint of interest. Before that our major focus will be on all venues present in and around the various regions of Banglore city.
Just a heads up on how many hotels are distributed around each region of Banglore, I have used Fousqaure API various venues specially food joint present in the region. On furthur notebook we will use Foursquare data to determine other venues as well.

#### Importing supporting libraries

In [11]:
# importing supporting libraries
import folium
import pandas as pd
import numpy as np
# for scraping a webpage
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
print('Libraries and packages imported!')


Libraries and packages imported!


In [2]:
# Scraping a wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore'
html_file = requests.get(url).text
soup = BeautifulSoup(html_file,'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of neighbourhoods in Bangalore - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XqvfeQpAAEEAAF7VpCoAAABI","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_neighbourhoods_in_Bangalore","wgTitle":"List of neighbourhoods in Bangalore","wgCurRevisionId":948839980,"wgRevisionId":948839980,"wgArticleId":53117184,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from August 2017","Use Indian English from August 2017","All Wikipedia articles wr

#### Finding various region from Neighborhood data of Banglore

In [3]:
Region = []
for reg in soup.find_all('span',class_ = 'mw-headline'):
        Region.append(reg.text)
del Region[8:11]
print(Region)

['Central', 'Eastern', 'North-Eastern', 'Northern', 'South-Eastern', 'Southern', 'Southern suburbs', 'Western']


#### Counting number of Neighborhoods

In [4]:
#counting number of Neighborhoods
dfreg = []
dfarea = []
i = 0
for area1 in soup.find_all('table',class_='wikitable sortable'):
    for area2 in area1.find_all('td'):
        dfarea.append(area2.text.strip())
        dfreg.append(Region[i])
    i = i+1
dfreg = dfreg[::3]        
dfarea = dfarea[::3]
print(len(dfreg))
print(len(dfarea))

65
65


#### Creating Dataframe of Regions and Neighborhoods

In [5]:
list_df = ['Region','Neighborhood']
df = pd.DataFrame(columns = list_df)
df['Region'] = dfreg
df['Neighborhood'] = dfarea
#pd.set_option("display.max_rows", None, "display.max_columns", None)
df.head()

,Region,Neighborhood
0,Central,Cantonment area
1,Central,Domlur
2,Central,Indiranagar
3,Central,Jeevanbheemanagar
4,Central,Malleswaram


#### Lat Long coordinates of Neighborhood

In [14]:
lat_long = ['Latitude', 'Longitude']
banglore_csv = 'Bang_Coords.csv'
banglore_coord = pd.read_csv(banglore_csv, names = lat_long)
banglore_coord.head()

,Latitude,Longitude
0,12.9936,77.5981
1,12.9610,77.6387
2,12.9784,77.6408
3,12.9642,77.6581
4,12.0055,77.5692


#### Merging Region, Neighborhood with Lat Long coords

In [15]:
df_lat_long = df.join(banglore_coord)
df_lat_long.head()

,Region,Neighborhood,Latitude,Longitude
0,Central,Cantonment area,12.9936,77.5981
1,Central,Domlur,12.9610,77.6387
2,Central,Indiranagar,12.9784,77.6408
3,Central,Jeevanbheemanagar,12.9642,77.6581
4,Central,Malleswaram,12.0055,77.5692


#### Obtaining Lat Long coords of Banglore city

In [16]:
city = 'Banglore, Karnataka'
geolocator = Nominatim(user_agent="Beng")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Banglore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Banglore are 13.0240558, 77.6240521.


#### Creating map of Bengaluru using latitude and longitude values

In [17]:
map_banglore = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dff['Latitude'], dff['Longitude'], dff['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bengaluru)  
    
map_banglore

# Detailed analysis for finding various venues in neighborhood and finding the best location for setting up a food joint is in the next notebook- 
# Battle of Neighborhood- Final